# 행정안정부 도로명주소 API

In [1]:
import pandas as pd
import numpy as np

# A simple, yet elegant HTTP library.
import requests
from urllib.parse import urlparse, quote

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm
import time

In [2]:
key_fd = open('config/roadapikey.txt', mode='r')
API_KEY = key_fd.read(100)
key_fd.close()

In [3]:
read_df = pd.read_csv('data/coffeeshop.csv', encoding='euc-kr')
read_df.shape

(93253, 39)

In [4]:
roadaddr = '서울특별시 강서구'
cafes = '스타벅스|이디야|빽다방|투썸'
df = read_df.query("도로명주소.str.contains(@roadaddr)", engine='python') \
    .query("상호명.str.contains(@cafes)", engine='python')
df.index = np.arange(0, len(df))
df.groupby('상호명').count().head()

,상가업소번호,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
상호명,,,,,,,,,,,,,,,,,,,,,
더본코리아_빽다방등촌,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
마스터키이디야마곡나루역점,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
빽다방,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,0,5,0,5,5
빽다방-목동사거리점,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,1
빽다방방화사거리점,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,1


In [5]:
addrs = []
for i in tqdm(df.index):
    try:
        url = "https://www.juso.go.kr/addrlink/addrLinkApi.do"
        params = { 
            'confmKey': API_KEY, 
            'currentPage': 1, 
            'countPerPage': 5, 
            'keyword': df['도로명주소'][i], 
            'resultType': 'json' 
        }
        api = requests.get(url, params=params).json()
        addrs.append(api['results']['juso'][0])
    except Exception as e:
        print(i, e)

100%|██████████| 77/77 [00:27<00:00,  2.82it/s]


In [6]:
df = pd.DataFrame(addrs)
df.to_csv('data/roadapi.csv', sep=',', encoding='utf8', index=False)